
---

# 🧠 Caching, ETags & Conditional Responses

> **Intent** → Reduce bandwidth and latency by letting clients reuse cached responses when data hasn’t changed.

---

## 🏷️ Core Ideas

* **Cache-Control**: tells clients/proxies how long to reuse a response (`max-age`, `public/private`).
* **ETag**: a version fingerprint of the resource (hash). If unchanged, return **304 Not Modified**.
* **Conditional requests**: clients send `If-None-Match` (ETag) or `If-Modified-Since` → server decides 200 vs 304.

---

## 🧾 Headers You’ll Use

* **Response**:

  * `ETag`: `"abc123"` (strong) or `W/"abc123"` (weak)
  * `Cache-Control`: `public, max-age=60` (or `private` for user-specific)
  * `Last-Modified`: HTTP date of last change
  * `Vary`: `Accept, Accept-Encoding, Authorization` (when output changes by header)
* **Request**:

  * `If-None-Match`: send back ETag from previous response
  * `If-Modified-Since`: send back Last-Modified

---

## 🧪 ETag Strategy

* **Strong ETag**: changes on any byte change (use when exact content matters).
* **Weak ETag**: changes on “meaningful” changes (safe for semantically same output).
* **Generation**: hash of canonicalized body or version field (e.g., `updated_at` + id).
* **Compare**: if client’s ETag matches current → respond **304** with empty body.

---

## 🕒 Cache-Control Strategy

* **Public data** (catalog, docs): `public, max-age=300`
* **User-specific** (requires auth): `private, max-age=60`
* **Rapidly changing**: short `max-age`, rely on ETags for freshness
* **Never cache**: `no-store` for sensitive endpoints (tokens, payments)

---

## 🔁 Invalidation & Freshness

* **Version field**: bump on updates (ensures new ETag)
* **Surrogate keys** (CDN): purge related objects together
* **Conditional GET**: prefer 304 over recomputing heavy responses

---

## 🧯 304 Not Modified

* Return when resource **unchanged** since client’s validator
* Include **no body**, keep headers minimal (can repeat `Cache-Control`)
* Saves bandwidth and CPU; improves latency

---

## 🔐 Security & Correctness

* Don’t mark **private data** as `public`
* Use `Vary` when output depends on headers (e.g., `Authorization`, `Accept-Language`)
* Avoid caching **error responses** unless intentional

---

## 🚀 Advanced Options

* `stale-while-revalidate`: serve slightly stale while refreshing in background
* `stale-if-error`: serve stale on upstream failure
* Layer with **CDN** for global performance (respect validators)

---

## 🏁 Outcome

Your API delivers **fast, bandwidth-efficient** responses:

* Clients re-use cached data via **ETags/Last-Modified**
* Servers return **304** when nothing changed
* Clear **Cache-Control/Vary** policies keep data correct and secure.
